# Image Classification Template
**Author:** Peyman Kh

**Date:** YYYY-MM-DD  

**Task:** Image classification with TensorFlow/Keras  

**Dataset:** [Dataset Name or Link]  


In [2]:
# Print TensorFlow version
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# List available physical devices (GPU or CPU)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("Using GPU:", gpus)
else:
    print("Using CPU.")

TensorFlow version: 2.18.0
Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Imports and setup

In [11]:
# Import necessary libraries
import os
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing import image_dataset_from_directory

AUTOTUNE = tf.data.AUTOTUNE

In [6]:
# Set random seed
def set_seed(seed=42):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    print(f"Random seeds set to {seed}.")

set_seed(42)

Random seeds set to 42.


# Config & Hyperparameters

In [8]:
class Config:
    DATA_DIR = 'data/dataset_name'
    IMG_SIZE = (224, 224)
    BATCH_SIZE = 32
    EPOCHS = 20
    LR = 1e-4
    SEED = 42
    NUM_CLASSES = 2

# Data loading and preprocessing

In [9]:
train_ds = image_dataset_from_directory(
    Config.DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=Config.SEED,
    image_size=Config.IMG_SIZE,
    batch_size=Config.BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    Config.DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=Config.SEED,
    image_size=Config.IMG_SIZE,
    batch_size=Config.BATCH_SIZE
)

class_names = train_ds.class_names
print("Classes:", class_names)

# Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Model definition

In [10]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=Config.IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(Config.NUM_CLASSES, activation='softmax')
])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Compile & Train

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=Config.LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=Config.EPOCHS,
    callbacks=[early_stop]
)

# Evaluation & Visualization

In [ ]:
def plot_history(history):
    plt.figure(figsize=(10, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Val')
    plt.title('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title('Loss')
    plt.legend()

    plt.show()

plot_history(history)

# Confusion Matrix & Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns

# Generate predictions
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, xticklabels=class_names, yticklabels=class_names, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


# Saving & Loading Model

In [ ]:
model.save('model.h5')
# Load later:
# model = tf.keras.models.load_model('model.h5')

## ✅ Next Steps
- Unfreeze top layers of the base model (fine-tuning)
- Add more data augmentation
- Try different architectures (EfficientNet, ViT)
- Convert to TF Lite for deployment
